In [1]:
from rpy2.robjects.packages import SignatureTranslatedAnonymousPackage
from rpy2.robjects.conversion import localconverter
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2 import robjects
import pandas as pd
import psycopg2
import h2o

In [2]:
def get_best_models(area):
    try:
        connection = psycopg2.connect(
            database="postgres",
            user="postgres",
            password="1234567890",
            host="127.0.0.1",
            port="5432"
        )

        # Cria um cursor para executar consultas
        cursor = connection.cursor()

        # Executa a consulta
        query = f"""
                SELECT "Tipo", "Subtipo", "MinDocs", "RangeDocs", COUNT(*)
                FROM(
                    SELECT * FROM public.models
                    WHERE ("Subarea", "F1") IN (

                      SELECT "Subarea", MAX("F1") AS max_f1

                      FROM public.models
                      WHERE "Area" = '{area}' AND "Subtipo" != 'ranger'

                      GROUP BY "Subarea"

                    )
                    ORDER BY "F1" DESC
                ) as tabela

                GROUP BY "Tipo", "Subtipo", "MinDocs", "RangeDocs"
                ORDER BY COUNT(*) DESC
                LIMIT 3
                """
        cursor.execute(query)

        # Recupera os resultados da consulta como uma lista de tuplas
        results = cursor.fetchall()

        # Cria um DataFrame pandas com os resultados
        df_modelos = pd.DataFrame(results, columns=[desc[0] for desc in cursor.description])

        # Fecha o cursor e a conexão
        cursor.close()

    except psycopg2.Error as error:
        print("Erro ao conectar ao PostgreSQL:", error)

    finally:
        # Fecha a conexão com o banco de dados
        if 'connection' in locals():
            connection.close()
        
        return df_modelos

In [3]:
def get_indicadores(area, subarea, tipo, subtipo, mindocs, rangedocs):
    try:
        connection = psycopg2.connect(
            database="postgres",
            user="postgres",
            password="1234567890",
            host="127.0.0.1",
            port="5432"
        )

        # Cria um cursor para executar consultas
        cursor = connection.cursor()

        # Executa a consulta
        query = f"""
                SELECT * FROM public.models
                    where("Tipo" ='{tipo}' AND
                        "Subtipo" = '{subtipo}' AND
                        "MinDocs" = {mindocs} AND
                        "RangeDocs" =  '{rangedocs}' AND 
                "Subarea" = '{subarea}' AND
                "Area" = '{area}')
                """
        cursor.execute(query)

        # Recupera os resultados da consulta como uma lista de tuplas
        results = cursor.fetchall()

        # Cria um DataFrame pandas com os resultados
        df_indicadores = pd.DataFrame(results, columns=[desc[0] for desc in cursor.description])
        df_indicadores = df_indicadores.drop(['Tipo', 'Subtipo', 'MinDocs', 'RangeDocs', 'Subarea', 'Area'], axis=1)
        # Fecha o cursor e a conexão
        cursor.close()

    except psycopg2.Error as error:
        print("Erro ao conectar ao PostgreSQL:", error)

    finally:
        # Fecha a conexão com o banco de dados
        if 'connection' in locals():
            connection.close()
        
        return df_indicadores

In [4]:
def get_df_row(str_df, id_id):
    base = importr('base')  
    
    df = robjects.r['load'](str_df)
    df = base.get(df[0])

    with localconverter(robjects.default_converter + pandas2ri.converter):
      df = robjects.conversion.rpy2py(df)

    df = df.loc[[df_id]]
    del df['Status']

    with localconverter(robjects.default_converter + pandas2ri.converter):
          df = robjects.conversion.py2rpy(df)
    
    del base
    
    return df

In [5]:
def prevNN(abstract):
    localH2o = h2o.init(nthreads = -1)
    
    Modelo = h2o.load_model('Modelos/DeepLearning_model_R_1670582405235_1')
    prevNN = Modelo.predict(h2o.H2OFrame(abstract))
    
    return prevNN[0, 0]

In [6]:
def prevC50(dados, str_modelo):
    base = importr('base')
    C50 = importr('C50')
    
    c50_model = robjects.r['load'](str_modelo)
    c50_model = base.get(c50_model[0])
    
    prevC50 = C50.predict_C5_0(c50_model, newdata = dados)
    
    del base
    del C50
    del c50_model
    
    return prevC50.levels[prevC50[0]-1]

In [7]:
def prevRFranger(dados, str_modelo):
    base = importr('base')
    
    robjects.r['load'](str_modelo)
    
    predict_r_code = """
    predict_rf <- function(model, data) {
        library(ranger)
        predict(Modelo, data)
    }
    """
    predict_r = SignatureTranslatedAnonymousPackage(predict_r_code, "predict_rf")
    
    modelo = robjects.globalenv['Modelo']
    previsoes_r = predict_r.predict_rf(modelo, dados)
    previsoes = previsoes_r.rx2('predictions')
    
    del base
    del modelo
    del predict_r_code
    
    return previsoes.levels[previsoes[0]-1]

In [8]:
def prevRFtrad(dados, str_modelo):
    base = importr('base')  
    rf = importr('randomForest')
    
    robjects.r['load'](str_modelo)
    Modelo = robjects.r['Modelo']
    
    predict = robjects.r('predict')
    
    prevRF_trad = predict(Modelo, data = dados)
    
    del rf
    del base
    del Modelo
    del predict
    
    return prevRF_trad.levels[prevRF_trad[0]-1]

In [9]:
def prevRpart(dados, str_modelo):
    base = importr('base')  
    rpart= importr('rpart')
    
    robjects.r['load'](str_modelo)
    Modelo = robjects.r['Modelo']
    
    predict = robjects.r('predict')
    
    prevRpart = predict(Modelo, data = dados, type="class")
    
    del rpart
    del base
    del Modelo
    del predict
    
    prevRpart.levels[prevRpart[0]-1]

In [10]:
str_df = 'Modelos/df.100x1x100.Ocorrencias.Rdata'
df_id = 'eef474adc4c2d494dca53fa6b3bd8211'
entrada = get_df_row(str_df, df_id)

In [11]:
with localconverter(robjects.default_converter + pandas2ri.converter):
      entrada_h2o = robjects.conversion.rpy2py(entrada)

In [12]:
area = prevNN(entrada_h2o)

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


--------------------------  -----------------------------
H2O_cluster_uptime:         1 hour 4 mins
H2O_cluster_timezone:       America/Sao_Paulo
H2O_data_parsing_timezone:  UTC
H2O_cluster_version:        3.38.0.1
H2O_cluster_version_age:    8 months and 24 days !!!
H2O_cluster_name:           H2O_from_python_renan_dwkhva
H2O_cluster_total_nodes:    1
H2O_cluster_free_memory:    3.645 Gb
H2O_cluster_total_cores:    8
H2O_cluster_allowed_cores:  8
H2O_cluster_status:         locked, healthy
H2O_connection_url:         http://localhost:54321
H2O_connection_proxy:       {"http": null, "https": null}
H2O_internal_security:      False
Python_version:             3.10.6 final
--------------------------  -----------------------------

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%


In [13]:
modelos = get_best_models(area)

In [55]:
vetor_strings = []

In [56]:
for i in modelos.index:
    if(str(modelos['Tipo'][i]) == "RF" and str(modelos['Subtipo'][i]) == "ranger"):
        str_modelo = "Modelos/Modelo.RF.ranger.200x1x280.Ocorrencias.ResearchAreaSA.RData"
        
        prev_sub =  prevRFranger(entrada, str_modelo)
        vetor_strings.append(prev_sub)
    
    if(str(modelos['Tipo'][i]) == "RF" and str(modelos['Subtipo'][i]) == "trad"):
        str_modelo = "Modelos/Modelo.RF.trad.200x1x280.Ocorrencias.RData"
        prev_sub =  prevRFtrad(entrada, str_modelo)
        vetor_strings.append(prev_sub)
    
    if(str(modelos['Tipo'][i]) == "C50"):
        str_modelo = f"""Modelos/Modelo.C50.trad.100x1x100.Ocorrencias.ResearchAreaSA.{area}.RData"""
        prev_sub =  prevC50(entrada, str_modelo)
        vetor_strings.append(f"""c("{prev_sub}", "{area}")""")
        
    if(str(modelos['Tipo'][i]) == "RPART"):
        str_modelo = "Modelos/Modelo.Rpart.trad.200x1x320.Ocorrencias.RData"
        prev_sub =  prevRpart(entrada, str_modelo)
        vetor_strings.append(f"""c("{prev_sub}", "{area}")""")

In [57]:
f"""c("{prev_sub}", "{area}")"""

'c("ENFERMAGEM", "CIENCIAS DA SAUDE")'

In [58]:
vetor_strings

['c("DIREITO, ESTADO E CONSTITUICAO", "CIENCIAS SOCIAIS APLICADAS")',
 'c("ENFERMAGEM", "CIENCIAS DA SAUDE")',
 'c("ENFERMAGEM", "CIENCIAS DA SAUDE")']

In [59]:
modelos

Tipo Subtipo  MinDocs RangeDocs  count
0   RF    trad      200     1x280     86
1  C50    trad      200     1x280      6
2  C50    trad      200     1x320      5

In [60]:
area1 = vetor_strings[0].split('",')[1].strip()
subarea1 = vetor_strings[0].split('",')[0].strip()
tipo1 = modelos['Tipo'][0]
subtipo1 = modelos['Subtipo'][0]
mindocs1 = modelos['MinDocs'][0]
rangedocs1 = modelos['RangeDocs'][0]
    
area2 = vetor_strings[1].split('",')[1].strip()
subarea2 = vetor_strings[1].split('",')[0].strip()
tipo2 = modelos['Tipo'][1]
subtipo2 = modelos['Subtipo'][1]
mindocs2 = modelos['MinDocs'][1]
rangedocs2 = modelos['RangeDocs'][1]

area3 = vetor_strings[2].split('",')[1].strip()
subarea3 = vetor_strings[2].split('",')[0].strip()
tipo3 = modelos['Tipo'][2]
subtipo3 = modelos['Subtipo'][2]
mindocs3 = modelos['MinDocs'][2]
rangedocs3 = modelos['RangeDocs'][2]
    
indicadores1 = get_indicadores(area1, subarea1, tipo1, subtipo1, mindocs1, rangedocs1)
indicadores2 = get_indicadores(area2, subarea2, tipo2, subtipo2, mindocs2, rangedocs2)
indicadores3 = get_indicadores(area3, subarea3, tipo3, subtipo3, mindocs3, rangedocs3)
    
count_indicadores1 = (indicadores1 > indicadores2).sum()
count_indicadores2 = (indicadores2 > indicadores1).sum()

In [61]:
count_indicadores1_2 = (indicadores1 > indicadores2).sum()
count_indicadores1_3 = (indicadores1 > indicadores3).sum()
    
count_indicadores2_1 = (indicadores2 > indicadores1).sum()
count_indicadores2_3 = (indicadores2 > indicadores3).sum()
    
count_indicadores3_1 = (indicadores3 > indicadores1).sum()
count_indicadores3_2 = (indicadores3 > indicadores2).sum()

In [62]:
if vetor_strings[0] == vetor_strings[1] and vetor_strings[1] == vetor_strings[2]:
    print("As três strings são iguais: " + vetor_strings[0])
    
elif vetor_strings[0] == vetor_strings[1]:
    print("A primeira e segunda string são iguais.")
    
    if pd.Series((count_indicadores3_1 > count_indicadores1_3), (count_indicadores3_2 > count_indicadores2_3)).any():
        print(vetor_strings[2])
    else:
        print(vetor_strings[0])
    
elif vetor_strings[0] == vetor_strings[2]:
    print("A primeira e terceira string são iguais.")
    
    if pd.Series((count_indicadores2_1 > count_indicadores1_2), (count_indicadores2_3 > count_indicadores3_2)).any():
        print(vetor_strings[1])
    else:
        print(vetor_strings[0])
    
elif vetor_strings[1] == vetor_strings[2]:
    print("A segunda e terceira string são iguais.")
    
    if pd.Series((count_indicadores1_2 > count_indicadores2_1), (count_indicadores1_3 > count_indicadores3_1)).any():
        print(vetor_strings[0])
    else:
        print(vetor_strings[2])
        
else:
    print("As três strings são diferentes.")
    
    if pd.Series((count_indicadores1_2 > count_indicadores2_1), (count_indicadores1_3 > count_indicadores3_1)).all():
        print(vetor_strings[0])
    elif pd.Series((count_indicadores2_1 > count_indicadores1_2), (count_indicadores2_3 > count_indicadores3_2)).all():
        print(vetor_strings[1])
    elif pd.Series((count_indicadores3_1 > count_indicadores1_3),(count_indicadores3_2 > count_indicadores2_3)).all():
        print(vetor_strings[2])
    else:
        print(vetor_strings[0])

A segunda e terceira string são iguais.
c("ENFERMAGEM", "CIENCIAS DA SAUDE")
